In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from sklearn import datasets
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
digits = datasets.load_digits()
rng = np.random.RandomState(0)
indices = np.arange(len(digits.data))
rng.shuffle(indices)

print(indices)

X = digits.data[indices[:330]]
y = digits.target[indices[:330]]
images = digits.images[indices[:330]]

n_total_samples = len(y)
n_labeled_points = 10
max_iterations = 0

unlabeled_indices = np.arange(n_total_samples)[n_labeled_points:]
f = plt.figure()

print(unlabeled_indices)
print(type(digits.data))

for i in range(max_iterations):
    if len(unlabeled_indices) == 0:
        print("No unlabeled items left to label.")
        break
    y_train = np.copy(y)
    y_train[unlabeled_indices] = -1

    lp_model = label_propagation.LabelSpreading(gamma=0.25, max_iter=5)
    lp_model.fit(X, y_train)

    predicted_labels = lp_model.transduction_[unlabeled_indices]
    true_labels = y[unlabeled_indices]

    cm = confusion_matrix(true_labels, predicted_labels,
                          labels=lp_model.classes_)

    print("Iteration %i %s" % (i, 70 * "_"))
    print("Label Spreading model: %d labeled & %d unlabeled (%d total)"
          % (n_labeled_points, n_total_samples - n_labeled_points,
             n_total_samples))

    print(classification_report(true_labels, predicted_labels))

    print("Confusion matrix")
    print(cm)

    # compute the entropies of transduced label distributions
    pred_entropies = stats.distributions.entropy(
        lp_model.label_distributions_.T)

    # select up to 5 digit examples that the classifier is most uncertain about
    uncertainty_index = np.argsort(pred_entropies)[::-1]
    uncertainty_index = uncertainty_index[
        np.in1d(uncertainty_index, unlabeled_indices)][:5]
    
    # keep track of indices that we get labels for
    delete_indices = np.array([])

    # for more than 5 iterations, visualize the gain only on the first 5
    if i < 5:
        f.text(.05, (1 - (i + 1) * .183),
               "model %d\n\nfit with\n%d labels" %
               ((i + 1), i * 5 + 10), size=10)
    for index, image_index in enumerate(uncertainty_index):
        image = images[image_index]

        # for more than 5 iterations, visualize the gain only on the first 5
        if i < 5:
            sub = f.add_subplot(5, 5, index + 1 + (5 * i))
            sub.imshow(image, cmap=plt.cm.gray_r, interpolation='none')
            sub.set_title("predict: %i\ntrue: %i" % (
                lp_model.transduction_[image_index], y[image_index]), size=10)
            sub.axis('off')

        # labeling 5 points, remote from labeled set
        delete_index, = np.where(unlabeled_indices == image_index)
        delete_indices = np.concatenate((delete_indices, delete_index))

    unlabeled_indices = np.delete(unlabeled_indices, delete_indices)
    n_labeled_points += len(uncertainty_index)

f.suptitle("Active learning with Label Propagation.\nRows show 5 most "
           "uncertain labels to learn with the next model.", y=1.15)
plt.subplots_adjust(left=0.2, bottom=0.03, right=0.9, top=0.9, wspace=0.2,
                    hspace=0.85)
plt.show()

[1081 1707  927 ... 1653  559  684]
[ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207
 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225
 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243
 244 245 246 24

<Figure size 432x288 with 0 Axes>

In [63]:
from sklearn.model_selection import LeaveOneGroupOut
from pandas import DataFrame
import numpy as np
import pandas as pd

train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

X_all = X_train.append(X_test).values
y_all = y_train.append(y_test).values

groups = train_df.append(test_df)['name']

rng = np.random.RandomState(0)
indices = np.arange(len(X_all))
rng.shuffle(indices)

X = X_all[indices[:10000]]
y = y_all[indices[:10000]]

n_total_samples = len(y)
n_labeled_points = 5000
max_iterations = 5

unlabeled_indices = np.arange(n_total_samples)[n_labeled_points:]

print(unlabeled_indices)
print(len(y_all))

for i in range(max_iterations):
    if len(unlabeled_indices) == 0:
        print("No unlabeled items left to label.")
        break
    y_train = np.copy(y)
    y_train[unlabeled_indices] = -1

    lp_model = label_propagation.LabelPropagation(gamma=0.25, max_iter=10)
    lp_model.fit(X, y_train)

    predicted_labels = lp_model.transduction_[unlabeled_indices]
    true_labels = y[unlabeled_indices]

    cm = confusion_matrix(true_labels, predicted_labels,
                          labels=lp_model.classes_)

    print("Iteration %i %s" % (i, 70 * "_"))
    print("Label Spreading model: %d labeled & %d unlabeled (%d total)"
          % (n_labeled_points, n_total_samples - n_labeled_points,
             n_total_samples))

    print(classification_report(true_labels, predicted_labels))

    print("Confusion matrix")
    print(cm)

    # compute the entropies of transduced label distributions
    pred_entropies = stats.distributions.entropy(
        lp_model.label_distributions_.T)

    # select up to 5 digit examples that the classifier is most uncertain about
    uncertainty_index = np.argsort(pred_entropies)[::-1]
    uncertainty_index = uncertainty_index[
        np.in1d(uncertainty_index, unlabeled_indices)][:1000]
    
    # keep track of indices that we get labels for
    delete_indices = np.array([])

    for index, image_index in enumerate(uncertainty_index):
        # labeling 5 points, remote from labeled set
        delete_index, = np.where(unlabeled_indices == image_index)
        delete_indices = np.concatenate((delete_indices, delete_index))
        y[image_index] = lp_model.transduction_[image_index]

    unlabeled_indices = np.delete(unlabeled_indices, delete_indices)
    n_labeled_points += len(uncertainty_index)

[5000 5001 5002 ... 9997 9998 9999]
10983
Iteration 0 ______________________________________________________________________
Label Spreading model: 5000 labeled & 5000 unlabeled (10000 total)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1929
           1       0.46      1.00      0.63      2288
           2       0.00      0.00      0.00       380
           3       0.00      0.00      0.00       403

   micro avg       0.46      0.46      0.46      5000
   macro avg       0.11      0.25      0.16      5000
weighted avg       0.21      0.46      0.29      5000

Confusion matrix
[[   0 1929    0    0]
 [   0 2288    0    0]
 [   0  380    0    0]
 [   0  403    0    0]]


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:79: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


Iteration 1 ______________________________________________________________________
Label Spreading model: 6000 labeled & 4000 unlabeled (10000 total)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1663
           1       0.44      1.00      0.61      1774
           2       0.00      0.00      0.00       253
           3       0.00      0.00      0.00       310

   micro avg       0.44      0.44      0.44      4000
   macro avg       0.11      0.25      0.15      4000
weighted avg       0.20      0.44      0.27      4000

Confusion matrix
[[   0 1663    0    0]
 [   0 1774    0    0]
 [   0  253    0    0]
 [   0  310    0    0]]


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:79: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


Iteration 2 ______________________________________________________________________
Label Spreading model: 7000 labeled & 3000 unlabeled (10000 total)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1647
           1       0.32      1.00      0.49       967
           2       0.00      0.00      0.00       141
           3       0.00      0.00      0.00       245

   micro avg       0.32      0.32      0.32      3000
   macro avg       0.08      0.25      0.12      3000
weighted avg       0.10      0.32      0.16      3000

Confusion matrix
[[   0 1647    0    0]
 [   0  967    0    0]
 [   0  141    0    0]
 [   0  245    0    0]]


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:79: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


Iteration 3 ______________________________________________________________________
Label Spreading model: 8000 labeled & 2000 unlabeled (10000 total)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1155
           1       0.30      1.00      0.47       606
           2       0.00      0.00      0.00        79
           3       0.00      0.00      0.00       160

   micro avg       0.30      0.30      0.30      2000
   macro avg       0.08      0.25      0.12      2000
weighted avg       0.09      0.30      0.14      2000

Confusion matrix
[[   0 1155    0    0]
 [   0  606    0    0]
 [   0   79    0    0]
 [   0  160    0    0]]


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:79: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


Iteration 4 ______________________________________________________________________
Label Spreading model: 9000 labeled & 1000 unlabeled (10000 total)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       575
           1       0.27      1.00      0.43       274
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        96

   micro avg       0.27      0.27      0.27      1000
   macro avg       0.07      0.25      0.11      1000
weighted avg       0.08      0.27      0.12      1000

Confusion matrix
[[  0 575   0   0]
 [  0 274   0   0]
 [  0  55   0   0]
 [  0  96   0   0]]


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:79: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
